<a href="https://colab.research.google.com/github/Grelatif/Data_Science/blob/main/Attention_Tokenizer_and_Dataset_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

base course:
https://github.com/priyammaz/PyTorch-Adventures/blob/main/PyTorch%20for%20NLP/Seq2Seq%20for%20Neural%20Machine%20Translation/tokenizer.py

In [ ]:
# imports

import os
import glob
import argparse
from tokenizers import Tokenizer
from tokenizers.trainers import WordPieceTrainer
from tokenizers.models import WordPiece
from tokenizers import normalizers
from tokenizers.normalizers import NFC, Lowercase
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import decoders
from tokenizers.processors import TemplateProcessing

In [ ]:
# create our own french tokeniezr

#dict of special tokens
special_token_dict = {"unknown_token": "[UNK]",
                      "pad_token": "[PAD]",
                      "start_token": "[BOS]",
                      "end_token": "[EOS]"}

def french_tokenizer(path_to_data_root):
  tokenizer = Tokenizer(WordPiece()) #subwords tokenization (like for BERT)
  tokenizer.normalizer = normalizers.Sequence([NFC(), Lowercase()])
  tokenizer.pre_tokenizer = Whitespace()
  #NFC(): Normalisation Unicode NFC (Normalization Form Canonical Composition).
  #Elle convertit par exemple des caractères décomposés (e + accent) en leur forme composée (é).
  #normalizers: Définit une séquence de normalisation du texte en entrée.
  #pre_tokenizer: methode de pretokenization
  #Whitespace: Coupe le texte d’entrée en tokens en se basant sur les espaces blancs (comme les espaces, tabulations, sauts de ligne)

  # Find all Target Language Files (ours are french ending with fr) #
  # Ces fichiers serviront à entraîner le tokenizer.
  french_files = glob.glob(os.path.join(path_to_data_root, "**/*.en"), recursive=True)# google drive "French_tokenizer"

  # Train Tokenizer #
  # Définit le "trainer" pour WordPiece :
  # - vocab_size : taille du vocabulaire final (32 000 sous-mots)
  # - special_tokens : liste de tokens spéciaux comme [PAD], [CLS], etc., fournie par `special_token_dict`
  trainer = WordPieceTrainer(vocab_size=32000, special_tokens=list(special_token_dict.values()))
  # Entraîne le tokenizer sur les fichiers .fr collectés
  tokenizer.train(french_files, trainer)
  # Créer le répertoire si il n'existe pas
  output_dir = "trained_tokenizer"
  os.makedirs(output_dir, exist_ok=True)  # Crée le répertoire s'il n'existe pas
  #save tokenizer
  tokenizer.save("trained_tokenizer/french_wp.json")

  # Le tokenizer est mainetant fonctionnel


# Nous allons maintenant créer une classe qui permet l'utilisation  pratique de ce tokenizer
# Cette classe encapsule toutes les fonctionnalités utiles du tokenizer :
# Encodage de phrases en ids.
# Décodage d’ids vers texte.
# Gestion des tokens spéciaux.
# Troncature/padding automatique.

class FrenchTokenizer():

# This is just a wrapper on top of the trained tokenizer to put together all the functionality we need
# for encoding and decoding

  def __init__(self, path_to_vocab, truncate=False, max_length=512):#trunc: tronquera les séquences trop longues à max_length.

    self.path_to_vocab = path_to_vocab#chemin vers le fichier du tokenizer sauvegardé
    self.tokenizer = self.prepare_tokenizer()#methode (future): charger le tokenizer depuis le fichier JSON, et retourne l’objet tokenizer.
    self.vocab_size = len(self.tokenizer.get_vocab())#taille du vocab
    self.special_tokens_dict = {"[UNK]": self.tokenizer.token_to_id("[UNK]"),
                                "[PAD]": self.tokenizer.token_to_id("[PAD]"),
                                "[BOS]": self.tokenizer.token_to_id("[BOS]"),#to put at beginning of sentence
                                "[EOS]": self.tokenizer.token_to_id("[EOS]")}#to put at end of sentence

    self.post_processor = TemplateProcessing(#TemplateProcessing définit une structure (template)
            single="[BOS] $A [EOS]",         #que le tokenizer applique après la tokenisation brute, pour
            special_tokens=[                 #ajouter les bons tokens spéciaux selon une logique précise (single, pair).
                ("[EOS]", self.tokenizer.token_to_id("[EOS]")),
                ("[BOS]", self.tokenizer.token_to_id("[BOS]"))
            ]
        )#Il configure le post-processing du tokenizer, c’est-à-dire ce qui se passe après
         # la tokenisation, mais avant l’entrée dans le modèle.
         #En particulier ici : ajout automatique de tokens spéciaux [BOS] et [EOS] autour de la séquence.
         #Les modèles comme les seq2seq, les encodeurs-décodeurs, ou les modèles autoregressifs
         #ont souvent besoin de tokens [BOS]/[EOS].

    self.truncate = truncate
    if self.truncate:
        self.max_len = max_length - self.post_processor.num_special_tokens_to_add(is_pair=False)
        # max_length est la longueur totale autorisée en entrée du modèle (ex : 512 pour BERT).
        # Or, ton tokenizer ajoute automatiquement des tokens spéciaux ([BOS], [EOS]) via post_processor.
        # Ces tokens prennent de la place dans la séquence encodée.
        # Donc, tu dois réserver cette place en enlevant leur nombre du max_length.

  def prepare_tokenizer(self): #charger et configurer le tokenizer
      tokenizer = Tokenizer.from_file(self.path_to_vocab)#load from  json file (via Tokenizer method)
      tokenizer.decoder = decoders.WordPiece()#what decoder to use? => WordPiece
      return tokenizer  # decode est une méthode simple, qui ne nécessite pas de configuration spécifique,
                        # donc on l'inclut dans prepare
                        # À l’inverse, encode dépend du contexte (troncature, tokens spéciaux, format
                        # de retour...), donc il vaut mieux la gérer dans une méthode personnalisée
                        # Meme si in fine on peut (et on va) implémenter un decode() spécifique



# Implémentons encode et decode, qui forment l'interface principale pour convertir entre :
# Texte ↔︎ Séquences de tokens (ids)
# Cette interface gère aussi bien les cas individuels (une phrase) que les cas en batch (plusieurs phrases)
# Intègre les options comme la troncature, l’ajout automatique de tokens spéciaux ([BOS], [EOS])
# Simplifie l'utilisation pour un utilisateur final

  def encode(self, input):
    # Applique la troncature si elle est activée
    # Applique le post-processing : ajout des [BOS], [EOS] (via TemplateProcessing)
    # Extrait les IDs finaux (ce qu'on veut au final)
    # Pourquoi une fonction interne ?
    # Pour éviter de répéter le code entre les deux cas (texte simple ou batch)
    # Elle n’est utilisée que dans encode(), donc elle est définie à l’intérieur :
    # c’est une bonne pratique (scope local, lisibilité)

    def _parse_process_tokenized(tokenized):
        if self.truncate:
            tokenized.truncate(self.max_len, direction="right")
        tokenized = self.post_processor.process(tokenized)
        return tokenized.ids

    if isinstance(input, str):# isinstance teste input pour voir s'il c'est une chaine de caracteres
        tokenized = self.tokenizer.encode(input)
        tokenized = _parse_process_tokenized(tokenized)

    elif isinstance(input, (list, tuple)):#teste si c'est soit une liste soit un tuple
        tokenized = self.tokenizer.encode_batch(input)#built-in method() from tokenizers
        tokenized = [_parse_process_tokenized(t) for t in tokenized]
    return tokenized


  def decode(self, input, skip_special_tokens=True):
      if isinstance(input, list):

          if all(isinstance(item, list) for item in input):#teste si tous les elements de inputs sont des listes
              decoded = self.tokenizer.decode_batch(input, skip_special_tokens=skip_special_tokens)
          elif all(isinstance(item, int) for item in input):
              decoded = self.tokenizer.decode(input, skip_special_tokens=skip_special_tokens)
      return decoded






In [ ]:
# tokenizer.decoder vs decode() method:
#tokenizer.decoder = ...	Définit comment recoller les morceaux de texte à partir des sous-mots
#FrenchTokenizer.decode()	Implémente la logique complète de décodage (selon les cas d'entrée, options, etc.)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# instance
path_to_data = "/content/drive/Othercomputers/Mon ordinateur portable/Data Science/French_tokenizer/training_europarl"
os.listdir(path_to_data)# this is not french data so we need to upload real french data later, but now it's just for testing

french_tokenizer(path_to_data)
# fr_tok = FrenchTokenizer(path_to_data)


In [ ]:
path_to_vocab = "/content/trained_tokenizer/french_wp.json"

In [ ]:
tokenizer_fr = FrenchTokenizer(path_to_vocab=path_to_vocab)


In [ ]:
tokenizer_fr.encode("La médiathèque est tellement bruyante")


[2, 986, 10492, 10982, 1687, 290, 3583, 21706, 237, 986, 2899, 938, 3]

In [ ]:
tokenizer_fr.encode(["la média est nulle","l'arsenal c'est beaucoup mieux"])
# ca marche super bien!

[[2, 986, 10492, 10982, 239, 1205, 20489, 237, 3],
 [2, 52, 10, 16389, 43, 10, 1205, 22226, 2301, 5250, 9027, 31932, 276, 3]]

In [ ]:
tokenizer_fr.decode([[2, 986, 10492, 10982, 239, 1205, 41, 600, 2491, 3],[2,52, 10, 16389, 43, 10, 1205, 22226,239, 1205, 20489, 237,3]], skip_special_tokens=False)

['[BOS] la média est a chier [EOS]',
 "[BOS] l ' arsenal c ' est beaua est nulle [EOS]"]